# Pytorch - Redes Neuronales

In [1]:
import torch

In [2]:
#D_in is input dimension, H is hidden dimension, D_out is output dimension , class is 10
D_in, H, D_out = 784, 55, 25


model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

In [3]:
# Mueve el modelo a la GPU si está disponible
if torch.cuda.is_available():
    model.cuda()

In [4]:
import numpy as np

In [5]:
data = np.loadtxt("/home/dilpz/Documentos/SIS420-2024/datasets/sign_mnist_train.csv", delimiter=',',skiprows=1)
# print(data)
X, Y = data[:, 1:], data[:, 0]
print(X.shape)
print(Y.shape)

data = np.loadtxt("/home/dilpz/Documentos/SIS420-2024/datasets/sign_mnist_test.csv", delimiter=',',skiprows=1)
# print(data)
X_test, Y_test = data[:, 1:], data[:, 0]
print(X.shape)
print(Y.shape)

(27455, 784)
(27455,)
(27455, 784)
(27455,)


In [6]:
# normalización y split
X_train, X_test, y_train, y_test = X/ 255., X_test/ 255., Y.astype(int), Y_test.astype(int)

In [7]:
# Define la función de pérdida y el optimizador
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [8]:
# Convierte los datos a tensores y cópialos a la GPU si está disponible
X_train_t = torch.from_numpy(X_train).float()
y_train_t = torch.from_numpy(y_train).long()
X_test_t = torch.from_numpy(X_test).float()
y_test_t = torch.from_numpy(y_test).long()

In [9]:
if torch.cuda.is_available():
    X_train_t = X_train_t.cuda()
    y_train_t = y_train_t.cuda()
    X_test_t = X_test_t.cuda()
    y_test_t = y_test_t.cuda()

In [13]:
# Entrenamiento del modelo
epochs = 100
log_each = 1
l = []

In [14]:
for e in range(1, epochs + 1):
    # Forward
    y_pred = model(X_train_t)

    # Calcula la pérdida
    loss = criterion(y_pred, y_train_t)
    l.append(loss.item())

    # Pone a cero los gradientes
    optimizer.zero_grad()

    # Retropropagación (calcula los gradientes automáticamente)
    loss.backward()

    # Actualiza los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

Epoch 100/1000 Loss 2.49781
Epoch 200/1000 Loss 2.30267
Epoch 300/1000 Loss 2.14244
Epoch 400/1000 Loss 2.00695
Epoch 500/1000 Loss 1.89831
Epoch 600/1000 Loss 1.80581
Epoch 700/1000 Loss 1.72149
Epoch 800/1000 Loss 1.64419
Epoch 900/1000 Loss 1.57663
Epoch 1000/1000 Loss 1.51678


In [17]:
from sklearn.metrics import accuracy_score
#Evalua en el conjunto de entrenamiento
model.eval()
y_pred = model(X_train_t)
y_pred = torch.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_train_t.cpu().numpy(), y_pred.cpu().numpy())
print(f"Accuracy: {accuracy * 100:.2f}%")

# Evalúa el modelo en el conjunto de prueba
model.eval()
y_pred = model(X_test_t)
y_pred = torch.argmax(y_pred, axis=1)
accuracy = accuracy_score(y_test_t.cpu().numpy(), y_pred.cpu().numpy())
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 71.43%
Accuracy: 63.33%
